#Load Training Data

In [2]:
import os
import glob
import imageio
import numpy as np

# Input setup
# Prepare Data
dataset = "Data\DIV2K_train_HR\DIV2K_train_HR"
hr_image_filenames = os.listdir(dataset)  # print(hr_image_filenames)
hr_image_dir  = os.path.join(os.getcwd(), dataset)  # print(data_dir)
hr_image_data = glob.glob(os.path.join(hr_image_dir,"*.png")) # print(data)
dataset = "Data\DIV2K_train_LR_bicubic_X4\DIV2K_train_LR_bicubic\X4"
lr_image_filenames = os.listdir(dataset)  # print(hr_image_filenames)
lr_image_dir  = os.path.join(os.getcwd(), dataset)  # print(data_dir)
lr_image_data = glob.glob(os.path.join(lr_image_dir,"*.png")) # print(data)

hr_images = []
lr_images = []

print("# images:", len(hr_image_data))

for i in range(5): #range(len(data)):
    hr_image = imageio.imread(hr_image_data[i]).astype(np.float)
    hr_images.append(hr_image)
    lr_image = imageio.imread(lr_image_data[i]).astype(np.float)
    lr_images.append(lr_image)

print(hr_images[0].shape)
print(lr_images[0].shape)



# images: 800
(1404, 2040, 3)
(351, 510, 3)


#Define the Model

In [5]:
import tensorflow as tf
#import tensorlayer as tfl

# Generator Model
def SRGAN_GEN(input_image):
    mu = 0
    sigma = 0.1
    # Layer 1: Convolution Input =. Output=.
    conv1_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev = sigma))
    conv1_bias   = tf.Variable(tf.zeros(64))
    conv1_output = tf.nn.conv2d(input_image, conv1_kernel, strides=[1, 1, 1, 1], act=tf.nn.relu, padding='SAME') + conv1_bias
    conv1_act = tf.nn.relu(conv1_output)
    tmp_conv_act = conv1_act
    # B residual blocks
    for i in range(16):
        # Layer: Convolution
        tmp_conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
        tmp_conv_bias   = tf.Variable(tf.zeros(64))
        tmp_conv_output = tf.nn.conv2d(tmp_conv_act, tmp_conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + tmp_conv_bias       
        # Layer: Batch Normalization
        tmp_conv_mean, tmp_conv_var = tf.nn.moments(tmp_conv_output, [0])
        tmp_conv_scale = tf.Variable(tf.ones([64]))
        tmp_conv_beta  = tf.Variable(tf.zeros([64]))
        tmp_conv_bn = tf.nn.batch_normalization(tmp_conv_output, tmp_conv_mean, tmp_conv_var, tmp_conv_beta, tmp_conv_scale, epsilon)        
        # Layer: Convolution
        tmp_conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
        tmp_conv_bias   = tf.Variable(tf.zeros(64))
        tmp_conv_output = tf.nn.conv2d(tmp_conv_bn, tmp_conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + tmp_conv_bias       
        # Layer: Batch Normalization
        tmp_conv_mean, tmp_conv_var = tf.nn.moments(tmp_conv_output, [0])
        tmp_conv_scale = tf.Variable(tf.ones([64]))
        tmp_conv_beta  = tf.Variable(tf.zeros([64]))
        tmp_conv_bn = tf.nn.batch_normalization(tmp_conv_output, tmp_conv_mean, tmp_conv_var, tmp_conv_beta, tmp_conv_scale, epsilon)        
        # Layer: Elementwise Add
        tmp_eltwise_out = tf.math.add([tmp_conv_act, tmp_conv_bn])
        tmp_conv_act = tmp_eltwise_out
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(tmp_conv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Batch Normalization
    conv_mean, conv_var = tf.nn.moments(conv_output, [0])
    conv_scale = tf.Variable(tf.ones([64]))
    conv_beta  = tf.Variable(tf.zeros([64]))
    conv_bn = tf.nn.batch_normalization(conv_output, conv_mean, conv_var, conv_beta, conv_scale, epsilon)        
    # Layer: Eltwise operations
    eltwise_out = tf.math.add([conv_bn, tmp_conv_act])
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(tmp_conv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Sub Pixel Convolution
    subpixconv_out = tfl.layers.SubpixelConv2d(conv_output, scale=2, n_out_channel=None, act=tf.nn.relu)
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(subpixconv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    # Layer: Sub Pixel Convolution
    subpixconv_out = tfl.layers.SubpixelConv2d(conv_output, scale=2, n_out_channel=None, act=tf.nn.relu)
    
    # Layer: Convolution
    conv_kernel = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 64), mean = mu, stddev= sigma))
    conv_bias   = tf.Variable(tf.zeros(64))
    conv_output = tf.nn.conv2d(subpixconv_act, conv_kernel, strides=[1, 1, 1, 1], padding='SAME') + conv_bias       
    
    return conv_output


# Discriminator
def SRGAN_DIS(input_image):
    mu = 0
    sigma = 0.1
    # Layer 1: Convolution Input =. Output=.
    conv1_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 3, 64), mean = mu, stddev = sigma))
    conv1_bias   = tf.Variable(tf.zeros(64))
    conv1_output = tf.nn.conv2d(input_image, conv1_kernel, strides=[1, 1, 1, 1], act=None, padding='SAME') + conv1_bias
    conv1_act    = tf.nn.leaky_relu(conv1_output, alpha=0.01) 

    # Layer 2: Convolution Input =. Output=.
    conv2_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 64, 64), mean = mu, stddev = sigma))
    conv2_bias   = tf.Variable(tf.zeros(64))
    conv2_output = tf.nn.conv2d(input_image, conv2_kernel, strides=[1, 2, 2, 1], act=None, padding='SAME') + conv2_bias
    conv2_act    = tf.nn.leaky_relu(conv2_output, alpha=0.01)
    # Layer 2: Batch Normalization
    conv2_mean, conv2_var = tf.nn.moments(conv2_act, [0])
    conv2_scale = tf.Variable(tf.ones([128]))
    conv2_beta  = tf.Variable(tf.zeros([128]))
    conv2_bn    = tf.nn.batch_normalization(conv2_act, conv2_mean, conv2_var, conv2_beta, conv2_scale, epsilon)        
    
    # Layer 3: Convolution Input =. Output=.
    conv3_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 64, 128), mean = mu, stddev = sigma))
    conv3_bias   = tf.Variable(tf.zeros(128))
    conv3_output = tf.nn.conv2d(input_image, conv3_kernel, strides=[1, 1, 1, 1], act=None, padding='SAME') + conv2_bias
    conv3_act    = tf.nn.leaky_relu(conv3_output, alpha=0.01)
    # Layer 3: Batch Normalization
    conv3_mean, conv2_var = tf.nn.moments(conv3_act, [0])
    conv3_scale = tf.Variable(tf.ones([128]))
    conv3_beta  = tf.Variable(tf.zeros([128]))
    conv3_bn    = tf.nn.batch_normalization(conv3_act, conv3_mean, conv3_var, conv3_beta, conv3_scale, epsilon)        
    
    # Layer 4: Convolution Input =. Output=.
    conv4_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 128, 128), mean = mu, stddev = sigma))
    conv4_bias   = tf.Variable(tf.zeros(128))
    conv4_output = tf.nn.conv2d(input_image, conv4_kernel, strides=[1, 2, 2, 1], act=None, padding='SAME') + conv2_bias
    conv4_act    = tf.nn.leaky_relu(conv4_output, alpha=0.01)
    # Layer 4: Batch Normalization
    conv4_mean, conv2_var = tf.nn.moments(conv4_act, [0])
    conv4_scale = tf.Variable(tf.ones([128]))
    conv4_beta  = tf.Variable(tf.zeros([128]))
    conv4_bn    = tf.nn.batch_normalization(conv4_act, conv4_mean, conv4_var, conv4_beta, conv4_scale, epsilon)   
    
    # Layer 5: Convolution Input =. Output=.
    conv5_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 128, 256), mean = mu, stddev = sigma))
    conv5_bias   = tf.Variable(tf.zeros(256))
    conv5_output = tf.nn.conv2d(input_image, conv5_kernel, strides=[1, 1, 1, 1], act=None, padding='SAME') + conv5_bias
    conv5_act    = tf.nn.leaky_relu(conv5_output, alpha=0.01)
    # Layer 5: Batch Normalization
    conv5_mean, conv5_var = tf.nn.moments(conv5_act, [0])
    conv5_scale = tf.Variable(tf.ones([256]))
    conv5_beta  = tf.Variable(tf.zeros([256]))
    conv5_bn    = tf.nn.batch_normalization(conv5_act, conv5_mean, conv5_var, conv5_beta, conv5_scale, epsilon)   
    
    # Layer 6: Convolution Input =. Output=.
    conv6_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 256, 256), mean = mu, stddev = sigma))
    conv6_bias   = tf.Variable(tf.zeros(256))
    conv6_output = tf.nn.conv2d(input_image, conv5_kernel, strides=[1, 2, 2, 1], act=None, padding='SAME') + conv6_bias
    conv6_act    = tf.nn.leaky_relu(conv5_output, alpha=0.01)
    # Layer 6: Batch Normalization
    conv6_mean, conv6_var = tf.nn.moments(conv6_act, [0])
    conv6_scale = tf.Variable(tf.ones([256]))
    conv6_beta  = tf.Variable(tf.zeros([256]))
    conv6_bn    = tf.nn.batch_normalization(conv6_act, conv6_mean, conv6_var, conv6_beta, conv6_scale, epsilon)   
    
    # Layer 7: Convolution Input =. Output=.
    conv7_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 256, 512), mean = mu, stddev = sigma))
    conv7_bias   = tf.Variable(tf.zeros(512))
    conv7_output = tf.nn.conv2d(input_image, conv7_kernel, strides=[1, 1, 1, 1], act=None, padding='SAME') + conv7_bias
    conv7_act    = tf.nn.leaky_relu(conv7_output, alpha=0.01)
    # Layer 7: Batch Normalization
    conv7_mean, conv6_var = tf.nn.moments(conv7_act, [0])
    conv7_scale = tf.Variable(tf.ones([512]))
    conv7_beta  = tf.Variable(tf.zeros([512]))
    conv7_bn    = tf.nn.batch_normalization(conv7_act, conv7_mean, conv7_var, conv7_beta, conv7_scale, epsilon)
    
    # Layer 8: Convolution Input =. Output=.
    conv8_kernel = tf.Variable(tf.truncated_normal(shape=(4, 4, 512, 512), mean = mu, stddev = sigma))
    conv8_bias   = tf.Variable(tf.zeros(512))
    conv8_output = tf.nn.conv2d(input_image, conv8_kernel, strides=[1, 2, 2, 1], act=None, padding='SAME') + conv8_bias
    conv8_act    = tf.nn.leaky_relu(conv8_output, alpha=0.01)
    # Layer 8: Batch Normalization
    conv8_mean, conv8_var = tf.nn.moments(conv8_act, [0])
    conv8_scale = tf.Variable(tf.ones([512]))
    conv8_beta  = tf.Variable(tf.zeros([512]))
    conv8_bn    = tf.nn.batch_normalization(conv8_act, conv8_mean, conv8_var, conv8_beta, conv8_scale, epsilon)
    
    
    
    
    
    
    
    
    
    
    
    


In [ ]:
def SRGAN_d(input_images, is_train=True, reuse=False):
    w_init = tf.random_normal_initializer(stddev=0.02)
    b_init = None  # tf.constant_initializer(value=0.0)
    gamma_init = tf.random_normal_initializer(1., 0.02)
    df_dim = 64
    lrelu = lambda x: tl.act.lrelu(x, 0.2)
    with tf.variable_scope("SRGAN_d", reuse=reuse):
        tl.layers.set_name_reuse(reuse)
        net_in = InputLayer(input_images, name='input/images')
        net_h0 = Conv2d(net_in, df_dim, (4, 4), (2, 2), act=lrelu, padding='SAME', W_init=w_init, name='h0/c')

        net_h1 = Conv2d(net_h0, df_dim * 2, (4, 4), (2, 2), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h1/c')
        net_h1 = BatchNormLayer(net_h1, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h1/bn')
        net_h2 = Conv2d(net_h1, df_dim * 4, (4, 4), (2, 2), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h2/c')
        net_h2 = BatchNormLayer(net_h2, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h2/bn')
        net_h3 = Conv2d(net_h2, df_dim * 8, (4, 4), (2, 2), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h3/c')
        net_h3 = BatchNormLayer(net_h3, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h3/bn')
        net_h4 = Conv2d(net_h3, df_dim * 16, (4, 4), (2, 2), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h4/c')
        net_h4 = BatchNormLayer(net_h4, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h4/bn')
        net_h5 = Conv2d(net_h4, df_dim * 32, (4, 4), (2, 2), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h5/c')
        net_h5 = BatchNormLayer(net_h5, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h5/bn')
        net_h6 = Conv2d(net_h5, df_dim * 16, (1, 1), (1, 1), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h6/c')
        net_h6 = BatchNormLayer(net_h6, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='h6/bn')
        net_h7 = Conv2d(net_h6, df_dim * 8, (1, 1), (1, 1), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='h7/c')
        net_h7 = BatchNormLayer(net_h7, is_train=is_train, gamma_init=gamma_init, name='h7/bn')

        net = Conv2d(net_h7, df_dim * 2, (1, 1), (1, 1), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='res/c')
        net = BatchNormLayer(net, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='res/bn')
        net = Conv2d(net, df_dim * 2, (3, 3), (1, 1), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='res/c2')
        net = BatchNormLayer(net, act=lrelu, is_train=is_train, gamma_init=gamma_init, name='res/bn2')
        net = Conv2d(net, df_dim * 8, (3, 3), (1, 1), act=None, padding='SAME', W_init=w_init, b_init=b_init, name='res/c3')
        net = BatchNormLayer(net, is_train=is_train, gamma_init=gamma_init, name='res/bn3')
        net_h8 = ElementwiseLayer([net_h7, net], combine_fn=tf.add, name='res/add')
        net_h8.outputs = tl.act.lrelu(net_h8.outputs, 0.2)

        net_ho = FlattenLayer(net_h8, name='ho/flatten')
        net_ho = DenseLayer(net_ho, n_units=1, act=tf.identity, W_init=w_init, name='ho/dense')
        logits = net_ho.outputs
        net_ho.outputs = tf.nn.sigmoid(net_ho.outputs)

    return net_ho, logits